In [1]:
# !wget https://raw.githubusercontent.com/mesolitica/malaysian-dataset/master/llm-benchmark/tatabahasabm.tripod.com/quiz-tatabahasa.jsonl

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from transformers.trainer_utils import get_last_checkpoint

latest = get_last_checkpoint("fpf-7b-instructions-16k-call")

In [4]:
tokenizer = AutoTokenizer.from_pretrained(latest)
model = AutoModelForCausalLM.from_pretrained(
    latest, 
    use_flash_attention_2 = True, 
    torch_dtype = torch.float16,
    device_map="cuda:0"
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
from tqdm import tqdm
import os
import random
import json

In [6]:
questions = []
with open('quiz-tatabahasa.jsonl') as fopen:
    for no, l in enumerate(fopen):
        l = json.loads(l)
        soalan = [l['question']]
        jawapan = None
        for c, k in l['choices'].items():
            soalan.append(f"{c}. {k['text']}")
            if k['answer']:
                jawapan = c
        
        data = {
            'no': no,
            'objektif': 'Jawab soalan yang diberikan' if l['instruction'] is None else l['instruction'],
            'soalan': '\n'.join(soalan),
            'jawapan': jawapan,
        }
        questions.append(data)
len(questions)

349

In [7]:
arange = set(range(len(questions)))

In [8]:
def convert_prompt(row, answer = False):
    if answer:
        prompt = f"""
objektif: {row['objektif']}
soalan: {row['soalan']}
jawapan: {row['jawapan']}
    """
    else:
        prompt = f"""
objektif: {row['objektif']}
soalan: {row['soalan']}
jawapan:
    """
    return prompt.strip()

In [9]:
def parse_llama_chat(messages):

    system = messages[0]['content']
    user_query = messages[-1]['content']

    users, assistants = [], []
    for q in messages[1:-1]:
        if q['role'] == 'user':
            users.append(q['content'])
        elif q['role'] == 'assistant':
            assistants.append(q['content'])

    texts = [f'<s>[INST] <<SYS>>\n{system}\n<</SYS>>\n\n']
    for u, a in zip(users, assistants):
        texts.append(f'{u.strip()} [/INST] {a.strip()} </s><s>[INST] ')
    texts.append(f'{user_query.strip()} [/INST]')
    prompt = ''.join(texts).strip()
    return prompt

In [10]:
i = 1
prompts = []

prompts.append(convert_prompt(questions[i]))
prompt = '\n\n'.join(prompts).strip()
print(prompt)

objektif: Jawab soalan yang diberikan
soalan: ........... merahnya bibir budak perempuan itu !
A. Aduhai
B. Ah
C. Amboi
D. Nah
jawapan:


In [11]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': prompt}
]
prompt = parse_llama_chat(messages)
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
inputs

{'input_ids': tensor([[    1,   518, 25580, 29962,  3532, 14816, 29903,  6778,    13,  1450,
           557,   594,   284,   801,   319, 29902,   343,   574,   286,  1160,
         29884,   432,  1450,   370,  2377,  2883,   577,   284,   273,    13,
         29966,   829, 14816, 29903,  6778,    13,    13,   711,  9761,   361,
         29901,   435,  1450,   370,   577,   284,   273,   343,   574,   652,
           495,  7941,    13,   578,   284,   273, 29901, 29871, 11296,   856,
          2778,   801,  1460, 29874, 29727,   381,  8619,   557,   639,  3451,
         12323,   372, 29884,  1738,    13, 29909, 29889,   319,   700, 23535,
            13, 29933, 29889,  9070,    13, 29907, 29889,  1913,   833, 29875,
            13, 29928, 29889,   405,   801,    13, 29926,  1450, 21419, 29901,
           518, 29914, 25580, 29962]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [12]:
generate_kwargs = dict(
    inputs,
    max_new_tokens=100,
    top_p=0.95,
    top_k=50,
    temperature=0.1,
    do_sample=True,
    num_beams=1,
    repetition_penalty=1.05,
)
r = model.generate(**generate_kwargs)
# print(tokenizer.decode(r[0]).split('jawapan:')[1].strip().split())

In [13]:
tokenizer.decode(r[0])

'<s> [INST] <<SYS>>\nawak adalah AI yang mampu jawab segala soalan\n<</SYS>>\n\nobjektif: Jawab soalan yang diberikan\nsoalan: ........... merahnya bibir budak perempuan itu !\nA. Aduhai\nB. Ah\nC. Amboi\nD. Nah\njawapan: [/INST] A </s>'

In [14]:
for i in tqdm(range(len(questions))):
    
    prompts = []
    prompts.append(convert_prompt(questions[i]))
    prompt = '\n\n'.join(prompts)
    messages = [
        {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
        {'role': 'user', 'content': prompt}
    ]
    prompt = parse_llama_chat(messages)
    inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
    repeat = []
    for _ in range(5):
        try:
            generate_kwargs = dict(
                inputs,
                max_new_tokens=100,
                top_p=0.95,
                top_k=50,
                temperature=0.3,
                do_sample=True,
                num_beams=1,
                repetition_penalty=1.05,
            )
            r = model.generate(**generate_kwargs)
            splitted = tokenizer.decode(r[0]).split('[/INST]')[1].strip().replace('</s>', '').replace('.', '').replace(',', '').strip().split()
            splitted = [t for t in splitted if len(t) == 1]
            repeat.append(splitted[0])
    
        except Exception as e:
            pass
    
    questions[i]['output'] = repeat

100%|██████████| 349/349 [02:24<00:00,  2.41it/s]


In [15]:
def most_common(l):
    return max(set(l), key=l.count)

In [16]:
filtered = [q for q in questions if 'output' in q and len(q['output'])]
correct = 0
for q in filtered:
    correct += most_common(q['output']) == q['jawapan']
(correct / len(filtered)) * 100

55.014326647564474